# ARE YOU ALL STARS MATERIAL?

In this notebook, we will explore whether a player with certain stats is All-Stars material or not using a machine learning approach. We will start by gathering and preparing our dataset, which consists of comprehensive statistics of NBA players, both current and past. This data has been meticulously collected and verified from the official [NBA website](https://www.nba.com/).

We will then proceed with data preprocessing, feature selection, and model training to predict the likelihood of a player being an All-Star. Let's dive into the exciting world of basketball analytics and machine learning!

First of all, we import the necessary dependencies.

In [6]:
import numpy as np
import pandas as pd


Then, we clean up the data and make sure Python can read each category correctly. E.g., the player's height is still in feet and inches format, we'd have to make sure that Python can read it as inches (with integer type).

In [ ]:
all_stars_df = pd.read_csv('players_dataset/All_Stars.csv', header=0)


                       Name  ... PIE (Player Impact Estimate)
0              Jaylen Brown  ...                         11.3
1             Stephen Curry  ...                         13.6
2              Kevin Durant  ...                         14.8
3              James Harden  ...                         14.2
4              Kyrie Irving  ...                         12.8
5              LeBron James  ...                         17.2
6            Damian Lillard  ...                         14.7
7              Jayson Tatum  ...                         16.2
8             Anthony Davis  ...                         18.5
9             Jalen Brunson  ...                         13.9
10          Cade Cunningham  ...                         14.8
11          Anthony Edwards  ...                         13.8
12           Darius Garland  ...                         12.9
13              Tyler Herro  ...                         13.7
14        Jaren Jackson Jr.  ...                         13.3
15      

Create a function for calculating cosine similarity metric

In [ ]:
def cosine_similarity(values):
    # should be able to take in 6 values (or any number of values) and return the cosine similarity between them
    pass

Create a function for calculating euclidian distance metric

In [ ]:
def euclidian_distance(values):
    # should be able to take in 6 values (or any number of values) and return the euclidian distance between them
    pass